<a href="https://colab.research.google.com/github/OskarKrafft/Machine-Learning-Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unused Chunks

In [ ]:
# Data Exploration

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(eppes.dtypes)

## Set up WD

In [ ]:
# Mount GDrive to facilitate data import
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change working directory to project folder
%cd /content/drive/MyDrive/Colab Notebooks/Machine-Learning-Project

In [ ]:
# If Github repo is not yet cloned in Drive, use following code (just do once!):

# ! git clone https://github.com/OskarKrafft/Machine-Learning-Project.git

## Import libraries

In [166]:
# Install packages and import them

import pandas as pd
import pyreadstat

## Import data

In [167]:
eppes, meta = pyreadstat.read_sav('data/raw/1.0-EPPES.sav')
pd.set_option('display.max_columns', None)

# Data Cleaning

## Drop non-voters

In [168]:
# Drop observation of people that were not eligible to vote

eppes = eppes[eppes.qg1.isna() == False]

# Check for NAs in vote variable qg1

eppes['qg1'].isna().sum()

0

In [169]:
# Create frequency table of qg1

eppes["qg1"].value_counts()

1.0    15008
2.0    12456
Name: qg1, dtype: int64

## Drop problematic, non-complete or uninteresting columns

In [170]:
# Import Excel sheet containing column indeces to be dropped

columns_analysis = pd.read_excel(r'Drop_Columns.xlsx')
columns_analysis = columns_analysis.drop(columns_analysis.columns[[0]], axis = 1)

In [171]:
# Create list of indeces of columns to be dropped

indeces_to_drop = []

for i in range(872):
  if columns_analysis.iloc[i, 2] == False:
    indeces_to_drop.append(i)

In [172]:
# Drop columns by indeces_to_drop

eppes = eppes.drop(eppes.columns[indeces_to_drop], axis = 1)

## Deal with NAs

In [173]:
# Check for NAs in columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(eppes.isna().sum())

studyno1       0
studyno2       0
doi            0
version        0
edition        0
survey         0
caseid         0
uniqid         0
serialid       0
tnscntry       0
country        0
isocntry       0
q1.1           0
q1.2           0
q1.3           0
q1.4           0
q1.5           0
q1.6           0
q1.7           0
q1.8           0
q1.9           0
q1.10          0
q1.11          0
q1.12          0
q1.13          0
q1.14          0
q1.15          0
q1.16          0
q1.17          0
q1.18          0
q1.19          0
q1.20          0
q1.21          0
q1.22          0
q1.23          0
q1.24          0
q1.25          0
q1.26          0
q1.27          0
q1.28          0
d70            0
d71a_1         0
d71a_2         0
d71a_3         0
qa1a_1         0
qa1a_2         0
qa1a_5         0
qa1a_6         0
qa1a_7         0
qa2a_1         0
qa2a_2         0
qa2a_3         0
qa2a_4         0
qa2a_5         0
qa2a_6         0
qa2a_7         0
qa3a.1         0
qa3a.2         0
qa3a.3        

Result: d1, d2 & d40b contain NAs

### Next Steps

**d1**: 
*In political matters people talk of "the left" and "the right". How would you place your views on this scale?*

-> Change values of rows with d1 == NA to d1 == 12 (DK)

**d25**:
*Would you say you live in a...? Rural area or village, small or middle sized town, etc.*

-> Change values of rows with d25 == NA to d25 == 4 (DK)

**d40b**:
Could you tell me how many children less than 10 years old live in your household?

-> replace values with 0

Oskar: Is replacing them with cero really correct??

In [174]:
# Replace values of rows with d1 == NA with d1 == 12

eppes.loc[eppes['d1'].isna(), 'd1'] = 12

# Replace values of rows with d25 == NA with d25 == 4

eppes.loc[eppes['d25'].isna(), 'd25'] = 4

# Replace values of rows with d40b == NA with d40b == 0

eppes.loc[eppes['d40b'].isna(), 'd40b'] = 0

In [175]:
# Check for NAs in eppes
eppes.isna().sum().sum()

0

## Delete rows with NAs in ordinal variables

In [176]:
eppes.describe()

,studyno1,studyno2,edition,survey,caseid,uniqid,serialid,tnscntry,country,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,d71a_1,d71a_2,d71a_3,qa1a_1,qa1a_2,qa1a_5,qa1a_6,qa1a_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa7,qa8_1,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_11,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qf1,qf2,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_5,qa18a_6,qa19,qb3_1,qb3_2,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qg1,qg10,qg11_1,qg11_2,qg11_4,qg11_5,qg11_6,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg15,d1,d7,d11,d15a,d25,d40a,d40b,d43a,d43b,d46.8,d60,d62_1,d62_2,d63,d72_1,d72_2,d77
count,27464.0,27464.0,27464.0,27464.0,2.746400e+04,2.746400e+04,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.0

d1, d7 and d11 have values of 97/98?

In [177]:
eppes["d1"].value_counts()

5.0     6563
12.0    2714
3.0     2489
4.0     2398
6.0     2380
97.0    2336
7.0     2184
8.0     1843
1.0     1434
10.0    1380
2.0     1060
9.0      683
Name: d1, dtype: int64

In [178]:
eppes["d7"].value_counts()

1.0     7318
2.0     6711
9.0     4135
13.0    2367
5.0     2209
11.0    1517
6.0      958
12.0     600
10.0     411
14.0     407
15.0     258
7.0      163
3.0      161
4.0      114
8.0       69
97.0      66
Name: d7, dtype: int64

In [179]:
eppes["d11"].value_counts()

65.0    605
68.0    603
55.0    570
60.0    567
50.0    549
       ... 
96.0      5
93.0      5
95.0      3
98.0      2
97.0      1
Name: d11, Length: 84, dtype: int64

Explanation: 

- d1 and d7: DKs
- d11: true age, no changes necessary
- for all ordinal variables: Highest number stands for DK


In [180]:
columns_analysis["encoding"].value_counts()

no             732
categorical     92
ordinal         48
Name: encoding, dtype: int64

In [181]:
# Create list of names of ordinal columns 

col_names_ordinal = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'ordinal':
    col_names_ordinal.append(columns_analysis.iloc[i, 1])

print(col_names_ordinal)

['d70', 'qa1a_1', 'qa1a_2', 'qa1a_5', 'qa1a_6', 'qa1a_7', 'qa7', 'qa8_1', 'qa8_4', 'qa8_5', 'qa8_6', 'qa8_7', 'qa8_8', 'qa8_9', 'qa8_11', 'qa8_13', 'qa10_1', 'qa10_2', 'qa18a_1', 'qa18a_2', 'qa18a_3', 'qa19', 'qb3_1', 'qb3_2', 'qd1a_1', 'qd1a_2', 'qd1a_3', 'qd1a_4', 'qd2_1', 'qd2_2', 'qd2_3', 'qd8', 'qd10a', 'qd10b', 'qd12_1', 'qd12_2', 'qe2_1', 'qe2_2', 'qg10', 'qg11_2', 'qg11_6', 'd1', 'd62_1', 'd62_2', 'd63', 'd72_1', 'd72_2', 'd77']


In [182]:
for i in range(48):
  print(eppes[col_names_ordinal[i]].value_counts())

2.0    15674
1.0     6943
3.0     3859
4.0      878
5.0      110
Name: d70, dtype: int64
2.0    14008
3.0     8751
4.0     2198
1.0     1932
5.0      575
Name: qa1a_1, dtype: int64
2.0    12465
3.0     9057
4.0     2556
1.0     2490
5.0      896
Name: qa1a_2, dtype: int64
2.0    16003
3.0     5873
1.0     3685
4.0     1303
5.0      600
Name: qa1a_5, dtype: int64
2.0    11671
3.0     9031
4.0     2917
1.0     2640
5.0     1205
Name: qa1a_6, dtype: int64
2.0    13404
3.0     8226
1.0     2502
4.0     2126
5.0     1206
Name: qa1a_7, dtype: int64
2.0    10653
3.0    10358
4.0     3167
1.0     2072
5.0      895
6.0      319
Name: qa7, dtype: int64
2.0    14991
1.0     6121
3.0     3634
5.0     1964
4.0      754
Name: qa8_1, dtype: int64
2.0    13251
3.0     7127
1.0     3824
5.0     1798
4.0     1464
Name: qa8_4, dtype: int64
2.0    15463
1.0     7949
3.0     2271
5.0     1407
4.0      374
Name: qa8_5, dtype: int64
2.0    12762
1.0     7351
3.0     4499
5.0     1881
4.0      971
Name: qa8_6

In [183]:
# Drop DKs for ordinal variables

for i in range(48):
  eppes = eppes[eppes[col_names_ordinal[i]] != max(eppes[col_names_ordinal[i]])]

# Drop special DKs

eppes = eppes[eppes.qg11_2 < 5]
eppes = eppes[eppes.qg11_6 < 5]
eppes = eppes[eppes.d1 < 11]
eppes = eppes[eppes.d63 < 6]


In [184]:
for i in range(48):
  print(eppes[col_names_ordinal[i]].value_counts())

2.0    7017
1.0    3405
3.0    1459
4.0     306
Name: d70, dtype: int64
2.0    6672
3.0    3631
1.0    1061
4.0     823
Name: qa1a_1, dtype: int64
2.0    5828
3.0    3912
1.0    1441
4.0    1006
Name: qa1a_2, dtype: int64
2.0    7322
3.0    2355
1.0    2047
4.0     463
Name: qa1a_5, dtype: int64
2.0    5701
3.0    3813
1.0    1473
4.0    1200
Name: qa1a_6, dtype: int64
2.0    6286
3.0    3678
1.0    1328
4.0     895
Name: qa1a_7, dtype: int64
2.0    5342
3.0    4010
4.0    1366
1.0    1130
5.0     339
Name: qa7, dtype: int64
2.0    6880
1.0    3189
3.0    1792
4.0     326
Name: qa8_1, dtype: int64
2.0    6202
3.0    3480
1.0    1888
4.0     617
Name: qa8_4, dtype: int64
2.0    6867
1.0    4030
3.0    1123
4.0     167
Name: qa8_5, dtype: int64
2.0    6008
1.0    3498
3.0    2247
4.0     434
Name: qa8_6, dtype: int64
2.0    6050
1.0    4445
3.0    1441
4.0     251
Name: qa8_7, dtype: int64
2.0    5843
1.0    4417
3.0    1654
4.0     273
Name: qa8_8, dtype: int64
2.0    6552
1.0    3929
3

## Change Value type from float to object for categoricals

In [185]:
eppes.dtypes.value_counts()

float64    300
object       3
dtype: int64

In [186]:
# Create list of names of categorical columns 

col_names_categorical = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'categorical':
    col_names_categorical.append(columns_analysis.iloc[i, 1])

In [187]:
# Change datatype of categorical variables to object

eppes[col_names_categorical] = eppes[col_names_categorical].astype('object')

eppes.dtypes.value_counts()

float64    208
object      95
dtype: int64

In [188]:
eppes.describe()

,studyno1,studyno2,edition,survey,caseid,uniqid,serialid,tnscntry,country,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,qa1a_1,qa1a_2,qa1a_5,qa1a_6,qa1a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,qa7,qa8_1,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_11,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa18a_1,qa18a_2,qa18a_3,qa19,qb3_1,qb3_2,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd12_1,qd12_2,qe2_1,qe2_2,qg10,qg11_2,qg11_6,d1,d11,d40a,d40b,d46.8,d62_1,d62_2,d63,d72_1,d72_2,d77
count,12187.0,12187.0,12187.0,12187.0,1.218700e+04,1.218700e+04,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,121